## Example EISCAT 3D visualizer

This is an simplified example that only uses SORTS

In [1]:
#Setup widgets
from __future__ import print_function
import ipywidgets as widgets
from IPython.display import display

#setup basic plotting
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

import numpy as np

#Load SORTS
import sorts

print(f'Using SORTS: {sorts.__version__}')

Using SORTS: 4.0.0


In [2]:
END_T = 6*60 #minutes

lims = {
    'L': [50, 200, 0.1],
    'N': [4, 100, 1],
    'dwell': [0.05, 2.0, 0.05],
    'h': [80, 400, 10],
    'h_min': [60, 400, 10],
    'beams': [1, 20, 1],
    't': [0, END_T-10.0, 0.1], #minutes
    't_span': [0.05, 10.0, 0.05], #sec
    'view_range': [300, 3000, 10],
}

In [3]:
class E3DScheduler(sorts.scheduler.PointingSchedule, sorts.scheduler.ObservedParameters):

    def __init__(self, radar, END_T, profiler=None, logger=None, **kwargs):
        super().__init__(
            radar=radar, 
            logger=logger, 
            profiler=profiler,
        )
        self.END_T = END_T

        self.controllers = []

    def update(self, L, N, dwell, h, h_min, beams):

        scanner_plane = sorts.controller.Scanner(
            self.radar, 
            sorts.scans.Plane(
                x_offset=0, 
                altitude=h, 
                x_size=L, 
                y_size=L, 
                x_num=N, 
                y_num=N, 
                dwell=dwell,
            ), 
            t=np.arange(0, self.END_T*0.5, dwell), 
            r=np.linspace(h_min, h, num=beams), 
            as_altitude=True,
        )
        
        scanner_fan = sorts.controller.Scanner(
            self.radar, 
            sorts.scans.Fence(
                azimuth=90, 
                num=40, 
                dwell=0.1, 
                min_elevation=30,
            ), 
            t=np.arange(self.END_T*0.5 + dwell, self.END_T, dwell), 
            r=np.linspace(h_min, h, num=beams), 
            as_altitude=True,
        )

        self.controllers = [scanner_plane, scanner_fan]

    def get_controllers(self):
        return self.controllers



def calc_observation(e3d_sched, ax, t, t_span, view_range, plot_rx, L, N, dwell, h, h_min, beams):
    e3d_sched.update(L*1e3, N, dwell, h*1e3, h_min*1e3, beams)
    ax.clear()
    sorts.plotting.schedule_pointing(
        e3d_sched, 
        t0=t, 
        t1=t+t_span, 
        ax=ax, 
        plot_rx=plot_rx, 
        view_range=view_range*1e3,
    )
    return ax

In [4]:
eiscat3d = sorts.radars.eiscat3d
e3d_sched = E3DScheduler(eiscat3d, END_T)

In [5]:
#Setup all the matplotlib stuff
output = widgets.Output()
with output:
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')

def fun(**kw):
    calc_observation(e3d_sched, ax, **kw)

grid = widgets.GridspecLayout(8,2)

app = widgets.AppLayout(
    header=None,
    left_sidebar=None,
    center=output,
    right_sidebar=grid,
    footer=None,
    height="600px", 
    width="100%",
)

###
# CREATE ALL THE SLIDERS AND UI.... AAAAAARGH!
###
t = widgets.FloatSlider(min=lims['t'][0], max=lims['t'][1], step=lims['t'][2], description='Time [s]'); grid[1,-2] = t
t_cnt = widgets.FloatText(min=lims['t'][0], max=lims['t'][1], step=lims['t'][2], description='Time [s]'); grid[0,-1] = t_cnt
t_span = widgets.FloatSlider(min=lims['t_span'][0], max=lims['t_span'][1], step=lims['t_span'][2], description='Time span [s]'); grid[2,-2] = t_span
view_range = widgets.FloatSlider(min=lims['view_range'][0], max=lims['view_range'][1], step=lims['view_range'][2], description='Zoom [km]'); grid[1,-1] = view_range
L = widgets.FloatSlider(min=lims['L'][0], max=lims['L'][1], step=lims['L'][2], description='Side-length [km]'); grid[3,-2] = L
N = widgets.IntSlider(min=lims['N'][0], max=lims['N'][1], step=lims['N'][2], description='Samples/side [1]'); grid[3,-1] = N
dwell = widgets.FloatSlider(min=lims['dwell'][0], max=lims['dwell'][1], step=lims['dwell'][2], description='Dwell [s]'); grid[4,-2] = dwell
h = widgets.FloatSlider(min=lims['h'][0], max=lims['h'][1], step=lims['h'][2], description='Altitude [km]'); grid[4,-1] = h
h_min = widgets.FloatSlider(min=lims['h_min'][0], max=lims['h_min'][1], step=lims['h_min'][2], description='Min altitude [km]'); grid[5,-2] = h_min
beams = widgets.IntSlider(min=lims['beams'][0], max=lims['beams'][1], step=lims['beams'][2], description='Rx-beams [1]'); grid[5,-1] = beams
plot_rx = widgets.ToggleButton(value=True, description='Plot rx beams'); grid[6,-2] = plot_rx

#Relations
def h_on_value_change(change):
    new_h = change['new']
    h_min.max = new_h
h.observe(h_on_value_change, names='value')

tlink = widgets.jslink((t, 'value'), (t_cnt, 'value'))

#Cache interactive to start with
out = widgets.interactive_output(
    fun, 
    dict(
        t = t,
        t_span = t_span,
        L = L,
        N = N,
        dwell = dwell,
        h = h,
        h_min = h_min,
        beams = beams,
        plot_rx = plot_rx,
        view_range = view_range,
    ),
)

#Display control
display(app, out)

AppLayout(children=(GridspecLayout(children=(FloatSlider(value=0.0, description='Time [s]', layout=Layout(grid…

Output()